In [1]:
import os
%pip install python-dotenv
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
SERPER_API_KEY = os.environ.get('SERPER_API_KEY')

if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY is not set")
if not SERPER_API_KEY:
    raise ValueError("SERPER_API_KEY is not set")

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# EDA and data cleaning, saved as filtered_traffic_data.csv
import pandas as pd

df = pd.read_csv('transformed_traffic_data.csv')

drop_cols = [
    'uri', 'query_parameters', 'headless_user_agent', 'session_stats_events',
    'session_stats_unique_pages', 'total_session_time_minutes', 'ip_stats_events_30d',
    'ip_stats_unique_pages_30d', 'asn_report_name', 'asn_description',
    'referrer_original', 'previous_timestamp', 'last_ip_visit_timestamp', 'last_updated_date'
]
df = df.drop(columns=[col for col in drop_cols if col in df.columns])
df = df[df['match_company_id'].notna()]

df.to_csv('filtered_traffic_data.csv', index=False)
print(f"filtered saved as filtered_traffic_data.csv，left {len(df)} rows")

filtered saved as filtered_traffic_data.csv，left 127206 rows


In [ ]:
# Install dependencies if needed
# %pip install lxml beautifulsoup4 google-generativeai

import requests
from bs4 import BeautifulSoup
import google.generativeai as genai
import json
import random
import re
import os
import time
from urllib.parse import urljoin

# 1. Load your universal website category framework
with open("universal_website_category_framework.json", "r", encoding="utf-8") as f:
    framework = json.load(f)

section_subsections = []
for section, subs in framework.items():
    for sub in subs:
        section_subsections.append((section, sub))

# 2. Input company name or URL
def is_url(s):
    url_pattern = re.compile(
        r'^(http://|https://|www\.)|([a-zA-Z0-9-]+\.[a-zA-Z]{2,})'
    )
    return bool(url_pattern.match(s.strip()))

def normalize_url(url):
    url = url.strip()
    if url.startswith("http://") or url.startswith("https://"):
        return url
    elif url.startswith("www."):
        return "http://" + url
    else:
        return "http://" + url

# 3. Search for sitemap URL
def search_sitemap(company_name):
    url = "https://google.serper.dev/search"
    payload = json.dumps({"q": f"{company_name} sitemap"})
    headers = {
        'X-API-KEY': SERPER_API_KEY,
        'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    search_results = json.loads(response.text)
    candidate_urls = []
    if 'organic' in search_results and len(search_results['organic']) > 0:
        for result in search_results['organic']:
            candidate_urls.append(result['link'])
    suffixes = ['', '/sitemap.xml', '/site-map', '/sitemap/']
    for base_url in candidate_urls:
        for suffix in suffixes:
            test_url = base_url.rstrip('/') + suffix
            try:
                resp = requests.get(test_url, timeout=5)
                if resp.status_code == 200:
                    return test_url
            except Exception:
                continue
    return None

# 4. Fetch URLs from sitemap (your function, works for XML/HTML)
def fetch_urls_from_sitemap(sitemap_url, max_links=100):
    urls = []
    try:
        resp = requests.get(sitemap_url, timeout=10)
        if resp.status_code != 200:
            print(f"Failed to fetch {sitemap_url}")
            return urls
        content_type = resp.headers.get("Content-Type", "")
        text = resp.text
        # Try XML first
        if sitemap_url.endswith('.xml') or 'xml' in content_type:
            soup = BeautifulSoup(text, "lxml-xml")
            sitemap_tags = soup.find_all("sitemap")
            if sitemap_tags:
                # Sitemap index
                for sitemap in sitemap_tags:
                    loc = sitemap.find("loc")
                    if loc and len(urls) < max_links:
                        child_urls = fetch_urls_from_sitemap(loc.text, max_links=max_links-len(urls))
                        urls += child_urls
                        if len(urls) >= max_links:
                            break
            else:
                # Regular XML sitemap
                for loc in soup.find_all("loc"):
                    if len(urls) < max_links:
                        urls.append(loc.text)
                    else:
                        break
        else:
            # Try HTML sitemap
            soup = BeautifulSoup(text, "lxml")
            for a in soup.find_all("a", href=True):
                href = a['href']
                if href.startswith("http"):
                    urls.append(href)
                elif href.startswith("/"):
                    full_url = urljoin(sitemap_url, href)
                    urls.append(full_url)
                if len(urls) >= max_links:
                    break
    except Exception as e:
        print(f"Error fetching/parsing sitemap: {e}")
    return list(set(urls))

# 5. Prepare mapping examples from the universal framework
mapping_examples = []
for section, subs in framework.items():
    for sub in subs:
        mapping_examples.append(f"  '{section} > {sub}' -> section: {section}, subsection: {sub}")
    if not subs:
        mapping_examples.append(f"  '{section}' -> section: {section}, subsection: null")

# 6. LLM mapping function
def extract_json_from_text(text):
    text = text.strip()
    if text.startswith("```json"):
        text = text[7:]
    if text.startswith("```"):
        text = text[3:]
    if text.endswith("```"):
        text = text[:-3]
    matches = re.findall(r"\{[\s\S]*?\}", text)
    for m in matches:
        try:
            return json.loads(m)
        except Exception:
            continue
    try:
        return json.loads(text)
    except Exception:
        return None

#for privacy concern, the company example here is deleted
def llm_url_to_framework_category_batch(urls, section_subsections, model, max_retries=3):
    options_str = "\n".join([f"- {sec} / {sub}" for sec, sub in section_subsections])
    example = (
        '{\n'
        '  "https://www.xxxxxxx.com/about-us/index": ["About Us", "About Us"],\n'
        '  "https://www.xxxxxxx.com/investors/shareholders/registration-services": ["Investors", "Investor Relations"]\n'
        '}'
    )
    prompt = (
        "You are a website taxonomy expert. For the following URLs, "
        "assign each to the most appropriate (section, subsection) from the universal website category framework below. "
        "You MUST choose from the provided pairs, do NOT invent new ones. Do NOT use 'Other'.\n\n"
        "Valid (section, subsection) pairs:\n"
        f"{options_str}\n\n"
        "Output as a JSON object mapping each URL to a [section, subsection] list. Example:\n"
        f"{example}\n"
        "URL list:\n"
        + "\n".join([f"- {url}" for url in urls]) +
        "\n\nJSON:"
    )
    for attempt in range(max_retries):
        try:
            response = model.generate_content(prompt)
            text = response.text
            print("LLM raw output:\n", text)
            parsed = extract_json_from_text(text)
            if parsed:
                return parsed
            else:
                print("No JSON found in LLM response, retrying...")
        except Exception as e:
            print(f"Error: {e}, retrying...")
            time.sleep(3)
    return {}

# 7. Configure Gemini
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-2.0-flash')

BATCH_TEST = True

if BATCH_TEST:
    df = pd.read_csv('filtered_traffic_data.csv')
    df = df[df['url'].notna()]
    batch_size = 1000
    total = min(200000, len(df))
    BATCH_SIZE_PER_LLM = 20  

    for batch_idx, start in enumerate(range(0, total, batch_size)):
        end = min(start + batch_size, total)
        outname = f'filtered_traffic_data_with_category_batch{batch_idx+1}.csv'
        if os.path.exists(outname):
            print(f"Batch {batch_idx+1} exists, skipping...")
            continue
        df_sample = df.iloc[start:end]
        urls_to_classify = df_sample['url'].tolist()
        print(f"Batch {batch_idx+1}: classifying {len(urls_to_classify)} URLs")
        classified_urls = {}
        for i in range(0, len(urls_to_classify), BATCH_SIZE_PER_LLM):
            sub_urls = urls_to_classify[i:i+BATCH_SIZE_PER_LLM]
            result = llm_url_to_framework_category_batch(
                sub_urls, section_subsections, model
            )
            classified_urls.update(result)

        df_sample['section'] = df_sample['url'].map(lambda x: classified_urls.get(x, [None, None])[0])
        df_sample['sub-section'] = df_sample['url'].map(lambda x: classified_urls.get(x, [None, None])[1])
        df_sample.to_csv(outname, index=False)
        print(f"Saved batch {batch_idx+1} to {outname}")
else:
    user_input = input("enter company name or URL: ").strip()
    if is_url(user_input):
        urls_to_classify = [normalize_url(user_input)]
        safe_company_name = "single_url"
        print(f"if enter URL，directly classify：{urls_to_classify[0]}")
    else:
        company_name = user_input
        sitemap_url = search_sitemap(company_name)
        if not sitemap_url:
            print("Could not find sitemap URL.")
            exit()
        print("Sitemap URL:", sitemap_url)
        max_links = 100
        urls_to_classify = fetch_urls_from_sitemap(sitemap_url, max_links=max_links)
        print(f"Found {len(urls_to_classify)} URLs in sitemap.")
        safe_company_name = company_name.replace(" ", "_").replace("/", "_")
    classified_urls = llm_url_to_framework_category_single(
        urls_to_classify, section_subsections, model
    )
    print(f"Classified {len(classified_urls)} URLs.")
    for item in list(classified_urls.items())[:10]:
        print(item)
    filename = f"url_to_universal_category_{safe_company_name}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(classified_urls, f, indent=2, ensure_ascii=False)
    print(f"Saved to {filename}")

Batch 1 exists, skipping...
Batch 2 exists, skipping...
Batch 3 exists, skipping...
Batch 4 exists, skipping...
Batch 5 exists, skipping...
Batch 6 exists, skipping...
Batch 7 exists, skipping...
Batch 8 exists, skipping...
Batch 9 exists, skipping...
Batch 10 exists, skipping...
Batch 11 exists, skipping...
Batch 12 exists, skipping...
Batch 13 exists, skipping...
Batch 14 exists, skipping...
Batch 15 exists, skipping...
Batch 16 exists, skipping...
Batch 17 exists, skipping...
Batch 18 exists, skipping...
Batch 19 exists, skipping...
Batch 20 exists, skipping...
Batch 21 exists, skipping...
Batch 22 exists, skipping...
Batch 23 exists, skipping...
Batch 24 exists, skipping...
Batch 25 exists, skipping...
Batch 26 exists, skipping...
Batch 27 exists, skipping...
Batch 28 exists, skipping...
Batch 29 exists, skipping...
Batch 30 exists, skipping...
Batch 31 exists, skipping...
Batch 32 exists, skipping...
Batch 33 exists, skipping...
Batch 34 exists, skipping...
Batch 35 exists, skippi

In [ ]:
import pandas as pd
import glob
import os
import json


all_files = glob.glob("filtered_traffic_data_with_category_batch*.csv")
df_list = [pd.read_csv(f) for f in all_files]
df_merged = pd.concat(df_list, ignore_index=True)


mask_empty = df_merged['section'].isna() | (df_merged['section'] == '') | (df_merged['section'].astype(str).str.lower() == 'none')
urls_to_retry = df_merged.loc[mask_empty, 'url'].tolist()


if os.path.exists("new_sections_partial.json"):
    with open("new_sections_partial.json", "r", encoding="utf-8") as f:
        new_sections = json.load(f)
else:
    new_sections = {}


urls_to_retry = [url for url in urls_to_retry if url not in new_sections]

BATCH_SIZE_PER_LLM = 20
for i in range(0, len(urls_to_retry), BATCH_SIZE_PER_LLM):
    sub_urls = urls_to_retry[i:i+BATCH_SIZE_PER_LLM]
    result = llm_url_to_framework_category_batch(sub_urls, section_subsections, model)
    new_sections.update(result)
    
    with open("new_sections_partial.json", "w", encoding="utf-8") as f:
        json.dump(new_sections, f, ensure_ascii=False, indent=2)
    print(f"已处理 {i+BATCH_SIZE_PER_LLM} / {len(urls_to_retry)}")


def fill_section(row):
    if (pd.isna(row['section']) or row['section'] in ['', None, 'None']) and row['url'] in new_sections:
        return new_sections[row['url']][0]
    return row['section']

def fill_subsection(row):
    if (pd.isna(row['sub-section']) or row['sub-section'] in ['', None, 'None']) and row['url'] in new_sections:
        return new_sections[row['url']][1]
    return row['sub-section']

df_merged['section'] = df_merged.apply(fill_section, axis=1)
df_merged['sub-section'] = df_merged.apply(fill_subsection, axis=1)
df_merged.to_csv("all_traffic_data_merged.csv", index=False)
print("saved to all_traffic_data_merged.csv")

In [ ]:
import pandas as pd
%pip install openpyxl
df_merged = pd.read_csv('all_traffic_data_merged.csv')
df_visitor = pd.read_csv('visitor_attributes.csv')

visitor_cols = ['company_type-2', 'factset_industries_name', 'factset_sector_name', 'visitor_id', 'institution_id']
df_visitor_sub = df_visitor.dropna(subset=['company_type-2', 'factset_industries_name', 'factset_sector_name'])

df_merged = df_merged.merge(
    df_visitor_sub[visitor_cols],
    how='left',
    left_on='visitor_id',
    right_on='visitor_id'
)

df_inst = pd.read_excel('institution_id_attributes.xlsx')
df_inst.columns = df_inst.columns.str.strip()

df_merged['institution_id'] = df_merged['institution_id'].astype(str)
df_inst['InstitutionID'] = df_inst['InstitutionID'].astype(str)

df_merged = df_merged.merge(
    df_inst[['InstitutionID', 'Side', 'InstitutionType', 'Country', 'Region']],
    how='left',
    left_on='institution_id',
    right_on='InstitutionID'
)

df_merged['location_country'] = df_merged['Country']

df_merged = df_merged.drop(columns=['InstitutionID', 'institution_id', 'Country'])

df_merged.to_csv('all_traffic_data_merged_with_attributes.csv', index=False)
print("Saved to all_traffic_data_merged_with_attributes.csv")


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
Saved to all_traffic_data_merged_with_attributes.csv
